In [1]:
import numpy as np
import pandas as pd
import cProfile

In [5]:
df = pd.read_excel("clinics.xls", sheet_name = "Results")
df.head()

,bizID,bizCat,bizCatSub,bizName,bizAddr,bizCity,bizState,bizZip,bizPhone,bizFax,...,bizURL,locAreaCode,locFIPS,locTimeZone,locDST,locLat,locLong,locMSA,locPMSA,locCounty
0,1,Clinics,Clinics,Hino Ronald H MD,98-151 Pali Momi Street Suite 142,Aiea,HI,96701,(808)487-2477,NaN,...,NaN,808,15003,PST-2,N,21.3980,-157.8981,3320.0,NaN,Honolulu
1,2,Clinics,Clinics,Farmer Joesph F Md,1225 Breckenridge Drive,Little Rock,AR,72205,(501)225-2594,NaN,...,NaN,501,5119,CST,Y,34.7495,-92.3533,4400.0,NaN,Pulaski
2,3,Clinics,Clinics & Medical Centers,Najjar Fadi Md,1155 West Linda Avenue Suite B,Hermiston,OR,97838,(541)289-1122,NaN,...,NaN,541,41059,PST,Y,45.8456,-119.2817,NaN,NaN,Umatilla
3,4,Clinics,Clinics & Medical Centers,Kittson Memorial Upper Level Nursing Home,1010 South Birch Avenue,Hallock,MN,56728,(218)843-2525,NaN,...,NaN,218,27069,CST,Y,48.7954,-97.0090,NaN,NaN,Kittson
4,5,Clinics,Clinics & Medical Centers,Thompson Robert B Md,100 North Eagle Creek Drive,Lexington,KY,40509,(859)258-4000,NaN,...,www.lexingtonclinic.com,859,21067,EST,Y,37.9935,-84.3712,4280.0,NaN,Fayette


In [6]:
df.columns

Index(['bizID', 'bizCat', 'bizCatSub', 'bizName', 'bizAddr', 'bizCity',
       'bizState', 'bizZip', 'bizPhone', 'bizFax', 'bizEmail', 'bizURL',
       'locAreaCode', 'locFIPS', 'locTimeZone', 'locDST', 'locLat', 'locLong',
       'locMSA', 'locPMSA', 'locCounty'],
      dtype='object')

In [8]:
# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [9]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['locLat'], df.iloc[i]['locLong'])
        distance_list.append(d)
    return distance_list
cProfile.run("df['distance'] = haversine_looping(df)")

         10056 function calls (9864 primitive calls) in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.007    0.007 4035510852.py:1(haversine_looping)
       30    0.001    0.000    0.001    0.000 763073546.py:2(haversine)
        1    0.000    0.000    0.000    0.000 <frozen _collections_abc>:104(_check_methods)
        1    0.000    0.000    0.000    0.000 <frozen _collections_abc>:288(__subclasshook__)
        1    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
      2/1    0.000    0.000    0.009    0.009 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 __init__.py:226(compile)
        1    0.000    0.000    0.001    0.001 __init__.py:280(_compile)
        3    0.000    0.000    0.000    0.000 __init__.py:34(using_copy_on_write)
        1    0.000    0.000    0

In [11]:
%%timeit
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['locLat'], row['locLong']))
#cProfile.run("df['distance'] = haversine_series")
df['distance'] = haversine_series

1.71 ms ± 42.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['locLat'], row['locLong']), \
                          axis=1)

740 μs ± 29.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%%timeit
# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(40.671, -73.985, df['locLat'], df['locLong'])

1.01 ms ± 46.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
cProfile.run("df['distance'] = haversine(40.671, -73.985, df['locLat'], df['locLong'])")

         3260 function calls (3181 primitive calls) in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 763073546.py:2(haversine)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        6    0.000    0.000    0.000    0.000 __init__.py:34(using_copy_on_write)
        3    0.000    0.000    0.000    0.000 __init__.py:42(warn_copy_on_write)
       22    0.000    0.000    0.000    0.000 _ufunc_config.py:132(geterr)
       22    0.000    0.000    0.000    0.000 _ufunc_config.py:33(seterr)
       11    0.000    0.000    0.000    0.000 _ufunc_config.py:426(__init__)
       11    0.000    0.000    0.000    0.000 _ufunc_config.py:430(__enter__)
       11    0.000    0.000    0.000    0.000 _ufunc_config.py:435(__exit__)
       11    0.000    0.000    0.000    0.000 

In [17]:
%%timeit
# Vectorized implementation of Haversine applied on NumPy arrays
df['distance'] = haversine(40.671, -73.985, df['locLat'].values, df['locLong'].values)

105 μs ± 2.92 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
cProfile.run("df['distance'] = haversine(40.671, -73.985, df['locLat'].values, df['locLong'].values)")

         165 function calls (159 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 763073546.py:2(haversine)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        7    0.000    0.000    0.000    0.000 __init__.py:34(using_copy_on_write)
        3    0.000    0.000    0.000    0.000 __init__.py:42(warn_copy_on_write)
        1    0.000    0.000    0.000    0.000 base.py:3777(get_loc)
        3    0.000    0.000    0.000    0.000 base.py:5323(__contains__)
        1    0.000    0.000    0.000    0.000 base.py:6672(_maybe_cast_indexer)
        1    0.000    0.000    0.000    0.000 base.py:82(shape)
        3    0.000    0.000    0.000    0.000 base.py:84(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:909(__len__)
        2    0.000    0.000    0.000    0.000 blocks.py:249(external_values)
        1    0.000    0.000    